# Example of using OpenAI streaming with Guardrails


In [1]:
# Few imports and global variables
from rich import print
import guardrails as gd
import openai

openai.api_key = "<YOUR_OPENAI_API_KEY>"

### Define the prompt and output schema


In [2]:
from pydantic import BaseModel, Field
from typing import List
from guardrails.validators import ValidRange, ValidChoices


# Define the prompt
prompt = """
Given the following doctor's notes about a patient, please extract a dictionary that contains the patient's information.

${doctors_notes}

${gr.complete_json_suffix_v2}
"""

doctors_notes = """49 y/o Male with chronic macular rash to face &amp; hair, worse in beard, eyebrows &amp; nares.
Itchy, flaky, slightly scaly. Moderate response to OTC steroid cream"""


# Define the output schema using Pydantic
class Symptom(BaseModel):
    symptom: str = Field(description="Symptom that a patient is experiencing")
    affected_area: str = Field(
        description="What part of the body the symptom is affecting",
        validators=[ValidChoices(choices=["head", "neck", "chest"], on_fail="reask")],
    )


class Medication(BaseModel):
    medication: str = Field(description="Name of the medication the patient is taking")
    response: str = Field(description="How the patient is responding to the medication")


class PatientInfo(BaseModel):
    gender: str = Field(description="Patient's gender")
    age: int = Field(
        description="Patient's age", validators=[ValidRange(min=0, max=100)]
    )
    symptoms: List[Symptom] = Field(
        description="Symptoms that the patient is currently experiencing. Each symptom should be classified into  separate item in the list."
    )
    current_meds: List[Medication] = Field(
        description="Medications the patient is currently taking and their response"
    )

### Create the Guard object


In [3]:
guard = gd.Guard.from_pydantic(output_class=PatientInfo, prompt=prompt)

#### Example 1: No streaming

By default, the `stream` parameter is set to `False`


In [4]:
# Wrap the OpenAI API call with the `guard` object
raw_llm_output, validated_output = guard(
    openai.Completion.create,
    prompt_params={"doctors_notes": doctors_notes},
    engine="text-davinci-003",
    max_tokens=1024,
    temperature=0.3,
)

# Print the validated output from the LLM
print(validated_output)

/Users/karanacharya/guardrails-ai/guardrails/guardrails/prompt/instructions.py:34: UserWarning: Instructions do not have any variables, if you are migrating follow the new variable convention documented here: https://docs.guardrailsai.com/0-2-migration/
  warn(
/Users/karanacharya/guardrails-ai/guardrails/guardrails/prompt/prompt.py:25: UserWarning: Prompt does not have any variables, if you are migrating follow the new variable convention documented here: https://docs.guardrailsai.com/0-2-migration/
  warnings.warn(


{
    'gender': 'Male',
    'age': 49,
    'symptoms': [
        {'symptom': 'macular rash', 'affected_area': 'head'},
        {'symptom': 'Itchy, flaky, slightly scaly', 'affected_area': 'neck'}
    ],
    'current_meds': [{'medication': 'OTC steroid cream', 'response': 'Moderate response'}]
}

In [5]:
# Let's see the logs
print(guard.state.most_recent_call.tree)

Logs
├── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
│   │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
│   │ │                                                                                                         │ │
│   │ │ Given the following doctor's notes about a patient, please extract a dictionary that contains the       │ │
│   │ │ patient's information.                                                                                  │ │
│   │ │                                                                                                         │ │
│   │ │ 49 y/o Male with chronic macular rash to face &amp; hair, worse in beard, eyebrows &amp; nares.         │ │
│   │ │ Itchy, flaky, slightly scaly. Moderate response to OTC steroid cream                                    │ │
│   │ │                                                                                                         │ │
│   │ │                                                                                                         │ │
│   │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
│   │ │ it into.                                                                                                │ │
│   │ │                                                                                                         │ │
│   │ │ <output>                                                                                                │ │
│   │ │     <string name="gender" description="Patient's gender"/>                                              │ │
│   │ │     <integer name="age" description="Patient's age" format="valid-range: min=0 max=100"/>               │ │
│   │ │     <list name="symptoms" description="Symptoms that the patient is currently experiencing. Each        │ │
│   │ │ symptom should be classified into  separate item in the list.">                                         │ │
│   │ │         <object>                                                                                        │ │
│   │ │             <string name="symptom" description="Symptom that a patient is experiencing"/>               │ │
│   │ │             <string name="affected_area" description="What part of the body the symptom is affecting"   │ │
│   │ │ format="valid-choices: choices=['head', 'neck', 'chest']"/>                                             │ │
│   │ │         </object>                                                                                       │ │
│   │ │     </list>                                                                                             │ │
│   │ │     <list name="current_meds" description="Medications the patient is currently taking and their        │ │
│   │ │ response">                                                                                              │ │
│   │ │         <object>                                                                                        │ │
│   │ │             <string name="medication" description="Name of the medication the patient is taking"/>      │ │
│   │ │             <string name="response" description="How the patient is responding to the medication"/>     │ │
│   │ │         </object>                                                                                       │ │
│   │ │     </list>                                                                                             │ │
│   │ │ </output>                                                                                               │ │
│   │ │                                                                                                         │ │
│   │ │                                                                                                         │ │
│   │ │ ONLY return a valid JSON object (no other t

#### Example 2: Streaming

Set the `stream` parameter to `True`


In [6]:
# Wrap the OpenAI API call with the `guard` object
raw_llm_output, validated_output = guard(
    openai.Completion.create,
    prompt_params={"doctors_notes": doctors_notes},
    engine="text-davinci-003",
    max_tokens=1024,
    temperature=0.3,
    stream=True,  # stream the output
)

# Print the validated output from the LLM
print(validated_output)

{
    'gender': 'Male',
    'age': 49,
    'symptoms': [
        {'symptom': 'macular rash', 'affected_area': 'head'},
        {'symptom': 'Itchy, flaky, slightly scaly', 'affected_area': 'neck'}
    ],
    'current_meds': [{'medication': 'OTC steroid cream', 'response': 'Moderate response'}]
}

In [7]:
# Let's see the logs
print(guard.state.most_recent_call.tree)

Logs
├── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
│   │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
│   │ │                                                                                                         │ │
│   │ │ Given the following doctor's notes about a patient, please extract a dictionary that contains the       │ │
│   │ │ patient's information.                                                                                  │ │
│   │ │                                                                                                         │ │
│   │ │ 49 y/o Male with chronic macular rash to face &amp; hair, worse in beard, eyebrows &amp; nares.         │ │
│   │ │ Itchy, flaky, slightly scaly. Moderate response to OTC steroid cream                                    │ │
│   │ │                                                                                                         │ │
│   │ │                                                                                                         │ │
│   │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
│   │ │ it into.                                                                                                │ │
│   │ │                                                                                                         │ │
│   │ │ <output>                                                                                                │ │
│   │ │     <string name="gender" description="Patient's gender"/>                                              │ │
│   │ │     <integer name="age" description="Patient's age" format="valid-range: min=0 max=100"/>               │ │
│   │ │     <list name="symptoms" description="Symptoms that the patient is currently experiencing. Each        │ │
│   │ │ symptom should be classified into  separate item in the list.">                                         │ │
│   │ │         <object>                                                                                        │ │
│   │ │             <string name="symptom" description="Symptom that a patient is experiencing"/>               │ │
│   │ │             <string name="affected_area" description="What part of the body the symptom is affecting"   │ │
│   │ │ format="valid-choices: choices=['head', 'neck', 'chest']"/>                                             │ │
│   │ │         </object>                                                                                       │ │
│   │ │     </list>                                                                                             │ │
│   │ │     <list name="current_meds" description="Medications the patient is currently taking and their        │ │
│   │ │ response">                                                                                              │ │
│   │ │         <object>                                                                                        │ │
│   │ │             <string name="medication" description="Name of the medication the patient is taking"/>      │ │
│   │ │             <string name="response" description="How the patient is responding to the medication"/>     │ │
│   │ │         </object>                                                                                       │ │
│   │ │     </list>                                                                                             │ │
│   │ │ </output>                                                                                               │ │
│   │ │                                                                                                         │ │
│   │ │                                                                                                         │ │
│   │ │ ONLY return a valid JSON object (no other t

As you can see here, the outputs in both examples match. Currently when `stream` is set to `True`, we simply concatenate the outputs from each chunk and return them as a single string. In the future, we will add more options for how to handle streaming outputs. For example, we will add chunk-level validation.
